In [ ]:
import data_import
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
from scipy.ndimage.filters import gaussian_filter


plt.rcParams.update({'font.size': 20})
plt.rc('figure', figsize=(20, 10))

In [ ]:
def exec(dt=1e-2, output_dt=1e-1, tf=60*3, vz=3, nh = 10):

    command = f"java -cp target/Zombies-1.0.jar \
                -Ddt={dt} -DoutputDt={output_dt} -Dtf={tf} \
                -Dvz={vz} -Dnh={nh} \
                Main"
    proc = os.popen(command)
    proc.readlines()
    proc.close()
    data = data_import.Data("zombies.txt")
    return data

In [ ]:
results_c = {}
vz = np.arange(1,6,0.5)
iterations = 10
NH = 140

if os.path.isfile("results_c.pickle"):
    results_c = pickle.load(open("results_c.pickle", "rb"))
else:


    for v in vz:
        res_iterations = {}
        for iteration in range(iterations):
            data = exec(nh=NH, dt=1e-2, output_dt=1,vz=v)
            time = []
            zcount = []
            res = {}
            for t, z, h, data in data:
                time.append(t)
                zcount.append(z)
            res["time"] = time
            res["zcount"] = zcount
            res_iterations[iteration] = res
        results_c[v] = res_iterations
            



In [ ]:
iteration = 0
for v in vz:
    y = results_c[v][iteration]["zcount"]
    #if y[0] == y[-1]: continue
    y = gaussian_filter(y, sigma=3)/NH
    plt.plot(results_c[v][iteration]["time"], y, label=f"{v} Vdz")
plt.legend()
plt.xlabel("Tiempo (s)")
plt.ylabel("Fraccion de zombies")
plt.show()

In [ ]:
sigma = 5
iter=0
for v in vz:
    y = results_c[v][iter]["zcount"]
    #if y[0] == y[-1]: continue
    y = gaussian_filter(results_c[v][iter]["zcount"], sigma=sigma, order=1, mode="nearest")
    dt = results_c[v][iter]["time"][1] - results_c[v][iter]["time"][0]
    y = np.array(y)/dt
    plt.plot(results_c[v][iter]["time"], y, label=f"{v} Vdz")
plt.legend()
plt.ylabel("Velocidad de Contagio (z/s)")
plt.xlabel("Tiempo (s)")
plt.show()


In [ ]:
means = []
errors = []
for v in vz:
    values = []
    for iteration in range(iterations):
        delta = results_c[v][iteration]["time"][1]-results_c[v][iteration]["time"][0]
        dif =np.diff(results_c[v][iteration]["zcount"])/delta
        
        values.append(np.mean(dif))
    means.append(np.mean(values))
    errors.append(np.std(values))
plt.errorbar(vz, means, yerr=errors, fmt="o-", capsize=10)
plt.xlabel("Vdz")
plt.ylabel("Velocidad de contagio (z/s)")
plt.xticks(vz)
plt.grid()
plt.show()

In [ ]:
means = []
errors = []
percentage = 100
for v in vz:
    values = []
    for iteration in range(iterations):
        time = 0
        count = results_c[v][iteration]["zcount"]
        times = results_c[v][iteration]["time"]
        for t, z in zip(times,count):
            if z/NH > percentage/100:
                time = t
                break
        
        values.append(t)
    means.append(np.mean(values))
    errors.append(np.std(values)/np.sqrt(len(values)))
plt.errorbar(vz, means, yerr=errors, fmt="o-")
plt.xlabel("Vdz")
plt.ylabel(f"Tiempo hasta Fz {percentage}% (s)")
plt.xticks(vz)
plt.grid()
plt.show()

In [ ]:
pickle.dump(results_c, open("results_c.pickle", "wb"))